# 필수과제 2
- KNNImputer
- 결측값에 대한 대체 방법,  결측치 대체할 때 거리기반으로 들어간다 na값들이 거리기반으로 대체가 된다.
- 개념 정리해주시고 결측치 보간해서 실제 어떤 식으로 결측치가 대체 되는지 확인
- 캘리포니아데이터를 실제 na값을 최소 100 정도 만들어 주시고 그 값을
 1. 평균으로 대체
 2. 0으로 대체
 3. KNN 대체
- 실제 RMSE 값이 어떤 식으로 바뀌는지를 KNN모델로 비교 정리

## 1단계 : 데이터 로드 및 확인

In [145]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn import datasets
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

In [146]:
dataset = datasets.fetch_california_housing()
df = pd.DataFrame(dataset.data, columns=dataset.feature_names)
df['MedHouseValue'] = dataset.target # target label 추가
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseValue
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [147]:
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

1. MedInc : 중위 소득
2. HouseAge : 건물 연령
3. AveRooms : 평균 방 개수
4. AveBedrms : 평균 침실 개수
5. Population : 인구
6. AveOccup : 
7. Latitude : 경도
8. Longtitude : 위도

The target variable is the median house value for California districts,   
expressed in hundreds of thousands of dollars ($100,000).

In [148]:
df.info() # 데이터의 종합적인 정보 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MedInc         20640 non-null  float64
 1   HouseAge       20640 non-null  float64
 2   AveRooms       20640 non-null  float64
 3   AveBedrms      20640 non-null  float64
 4   Population     20640 non-null  float64
 5   AveOccup       20640 non-null  float64
 6   Latitude       20640 non-null  float64
 7   Longitude      20640 non-null  float64
 8   MedHouseValue  20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


In [149]:
df.isnull().sum() # 결측치 없음

MedInc           0
HouseAge         0
AveRooms         0
AveBedrms        0
Population       0
AveOccup         0
Latitude         0
Longitude        0
MedHouseValue    0
dtype: int64

In [150]:
df.duplicated().sum() # 중복 값 없음

0

In [151]:
df.describe() # 기술 통계량 확인

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704,2.068558
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532,1.153956
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000,0.149990
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000,1.196000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000,1.797000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000,2.647250
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000,5.000010


In [152]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseValue
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


## 2단계 :  결측치 만들기 

In [153]:
len(df)

20640

In [154]:
for i in range(0, len(df), 200):
    temp = df.iloc[i][8]
    df.iloc[i] = np.NaN
    df.iloc[i][8] = temp
df.isnull().sum() # 104 행이 결측치가 되었다

MedInc           104
HouseAge         104
AveRooms         104
AveBedrms        104
Population       104
AveOccup         104
Latitude         104
Longitude        104
MedHouseValue    104
dtype: int64

In [155]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseValue
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [156]:
# 각각의 데이터셋에 대해 RMSE를 비교하기 위해 데이터셋을 독립적으로 구분
df_mean = df.copy()
df_zero = df.copy()
df_KNN = df.copy()

## 3단계 : 결측치 대체 및 모델 학습 & 평가 

In [157]:
RMSE_list = [] # 결측치를 다르게 처리한 세 경우에 대한 RMSE를 저장할 리스트

In [158]:
def fit_predict_rmse(data, label, title):
    # 데이터셋 분리
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)

    # 모델과 하이퍼 파라미터 설정
    model = KNeighborsRegressor(algorithm='ball_tree', metric='manhattan', n_neighbors=11, weights='distance')

    # 훈련 데이터로 모델 학습
    model.fit(X_train, y_train) 

    # 학습이 된 모델을 통해 테스트 데이터를 바탕으로 예측
    y_pred = model.predict(X_test)

    # 예측값과 실제값 사이의 오차로 성능 평가
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print("RMSE: ", rmse)
    RMSE_list.append((title, rmse))

### 1) 평균으로 대체 

In [159]:
column_mean = df_mean.mean()
column_mean

MedInc              3.870761
HouseAge           28.640728
AveRooms            5.429256
AveBedrms           1.096774
Population       1426.074844
AveOccup            3.071627
Latitude           35.631868
Longitude        -119.569882
MedHouseValue       2.068776
dtype: float64

In [160]:
df_mean = df_mean.fillna(column_mean) # df_mean의 결측치를 각 열의 평균값으로 대체
df_mean.isnull().sum()

MedInc           0
HouseAge         0
AveRooms         0
AveBedrms        0
Population       0
AveOccup         0
Latitude         0
Longitude        0
MedHouseValue    0
dtype: int64

In [161]:
df_mean.iloc[0] == column_mean

MedInc           True
HouseAge         True
AveRooms         True
AveBedrms        True
Population       True
AveOccup         True
Latitude         True
Longitude        True
MedHouseValue    True
dtype: bool

In [162]:
data = df_mean.drop('MedHouseValue', axis=1)
label = df_mean.MedHouseValue

In [163]:
fit_predict_rmse(data, label, "MEAN")

RMSE:  0.9725032015586599


### 2) 0으로 대체

In [164]:
df_zero.isnull().sum() # df_mean과 독립적이기 때문에 df_mean을 수정한 이후에도 결측치가 유지되어 있다

MedInc           104
HouseAge         104
AveRooms         104
AveBedrms        104
Population       104
AveOccup         104
Latitude         104
Longitude        104
MedHouseValue    104
dtype: int64

In [165]:
df_zero = df_zero.fillna(0)
df_zero.isnull().sum()

MedInc           0
HouseAge         0
AveRooms         0
AveBedrms        0
Population       0
AveOccup         0
Latitude         0
Longitude        0
MedHouseValue    0
dtype: int64

In [166]:
df_zero.head() # 첫번째 행과 같이 NA값이 0으로 대체되어 있다

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseValue
0,0.0000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.00,0.000
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [167]:
data = df_zero.drop('MedHouseValue', axis=1)
label = df_zero.MedHouseValue
fit_predict_rmse(data, label, "ZERO")

RMSE:  0.9723471434403117


### 3) KNN 기반으로 대체(KNNImputer)

In [168]:
df_KNN.isnull().sum()

MedInc           104
HouseAge         104
AveRooms         104
AveBedrms        104
Population       104
AveOccup         104
Latitude         104
Longitude        104
MedHouseValue    104
dtype: int64

In [169]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=10)

# KNNImputer를 거치면 데이터가 ndarray가 되기 때문에 pd.DataFrame()을 다시 사용해야한다.
cols = df_KNN.columns # 이 때 사용할 column명들을 기존 데이터에서 뽑아서 cols변수에 담아놓는다
cols

Index(['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup',
       'Latitude', 'Longitude', 'MedHouseValue'],
      dtype='object')

In [170]:
data = df_KNN.drop('MedHouseValue', axis=1)
label = df_KNN.MedHouseValue
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)

In [171]:
df_KNN = imputer.fit_transform(df_KNN)

In [172]:
df_KNN = pd.DataFrame(df_KNN, columns=cols)
df_KNN.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseValue
0,3.870761,28.640728,5.429256,1.096774,1426.074844,3.071627,35.631868,-119.569882,2.068776
1,8.301400,21.000000,6.238137,0.971880,2401.000000,2.109842,37.860000,-122.220000,3.585000
2,7.257400,52.000000,8.288136,1.073446,496.000000,2.802260,37.850000,-122.240000,3.521000
3,5.643100,52.000000,5.817352,1.073059,558.000000,2.547945,37.850000,-122.250000,3.413000
4,3.846200,52.000000,6.281853,1.081081,565.000000,2.181467,37.850000,-122.250000,3.422000


In [173]:
data = df_KNN.drop('MedHouseValue', axis=1)
label = df_KNN.MedHouseValue
fit_predict_rmse(data, label, "KNN")

RMSE:  0.9725032015586599


In [174]:
RMSE_list

[('MEAN', 0.9725032015586599),
 ('ZERO', 0.9723471434403117),
 ('KNN', 0.9725032015586599)]